In [4]:
import numpy as np
import cv2
import torch
from pytorch_lightning import seed_everything
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Importing specific utilities
from src.test.test_codec import process
from utils.share import *
import utils.config as config
from annotator.util import resize_image, HWC3
from annotator.canny import CannyDetector
from models.util import create_model, load_state_dict
from models.ddim_hacked import DDIMSampler

In [5]:
ckpt_path= "/data/maryam.sana/vimeo_unicontrol/Uni-ControlNet/checkpoints/vimeo_8/uni_v8.ckpt"
config_path ="configs/uni_v15.yaml"
prompt = "A beautiful blonde girl with pink lipstick"
model = create_model(config_path).cpu()
model.load_state_dict(load_state_dict(ckpt_path, location="cuda"))
model = model.cuda()

a_prompt = "best quality, extremely detailed"
n_prompt = "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality"
num_samples = 1
image_resolution = 512
ddim_steps = 50
strength = 1
scale = 7.5
seed = 42
eta = 0.0
global_strength = 1

UniControlNet: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, conte

In [10]:
image_paths = sorted(glob.glob('../vimeo_unicontrol/Uni-ControlNet/data/UVG/Beauty/*.png'))[1:20]
optical_flows = sorted(glob.glob('../vimeo_unicontrol/Uni-ControlNet/data/UVG/optical_flow/Beauty_reconstructed/*.png'))
encoded_frame = cv2.imread("/data/maryam.sana/vimeo_unicontrol/Uni-ControlNet/data/UVG/Beauty/quality_4/im00001.png")  # Load encoded frame
# print(image_paths,optical_flows)
prompt = "A beautiful blonde girl with pink lipstick"

In [11]:
encoded_frame = cv2.cvtColor(encoded_frame,cv2.COLOR_BGR2RGB)
pred_folder = "test_codec/beauty/"
os.makedirs(pred_folder, exist_ok=True)

# Load images
original_images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]
optical_flows_images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in optical_flows]

# Initialize previous frame as the encoded frame for the second image
previous_frame = encoded_frame

# List to store predictions
predictions = []

# Process images
for i in range(len(image_paths)):
    original_image = original_images[i]
    canny_image = optical_flows_images[i]

    # Use optical flow for the second image and previous prediction for subsequent frames
    if i == 0:
        frame_image = previous_frame  # Use the encoded frame for the first image
    else:
        frame_image = predictions[-1]  # Use the last prediction for subsequent frames

    # Use the `process` function
    pred = process(
        model,
        canny_image,
        frame_image,
        prompt,
        a_prompt,
        n_prompt,
        num_samples,
        image_resolution,
        ddim_steps,
        strength,
        scale,
        seed,
        eta,
        global_strength,
    )
    pred_img = pred[0][0]

    # Ensure the images are of the same size
    if original_image.shape != pred_img.shape:
        pred_img = cv2.resize(pred_img, (original_image.shape[1], original_image.shape[0]))

    # Save the prediction for the next frame
    predictions.append(pred_img)

    # Save prediction image to disk
    pred_image_path = os.path.join(pred_folder, f"im{i + 1}_pred.png")
    cv2.imwrite(pred_image_path, cv2.cvtColor(pred_img, cv2.COLOR_RGB2BGR))
    print(f"Saved prediction image: {pred_image_path}")


Global seed set to 42


(512, 512, 3)
(512, 512, 3)
Selected timesteps for ddim sampler: [  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341
 361 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701
 721 741 761 781 801 821 841 861 881 901 921 941 961 981]
Selected alphas for ddim sampler: a_t: tensor([0.9983, 0.9804, 0.9609, 0.9398, 0.9171, 0.8930, 0.8674, 0.8404, 0.8121,
        0.7827, 0.7521, 0.7207, 0.6885, 0.6557, 0.6224, 0.5888, 0.5551, 0.5215,
        0.4882, 0.4552, 0.4229, 0.3913, 0.3605, 0.3308, 0.3023, 0.2750, 0.2490,
        0.2245, 0.2014, 0.1799, 0.1598, 0.1413, 0.1243, 0.1087, 0.0946, 0.0819,
        0.0705, 0.0604, 0.0514, 0.0435, 0.0365, 0.0305, 0.0254, 0.0210, 0.0172,
        0.0140, 0.0113, 0.0091, 0.0073, 0.0058]); a_(t-1): [0.99914998 0.99829602 0.98038077 0.96087277 0.93978298 0.91713792
 0.89298052 0.86737001 0.84038192 0.81210774 0.78265446 0.75214338
 0.72070938 0.68849909 0.65566933 0.62238538 0.58881873 0.55514455
 0.52153981 0.4881804  0.4552387

DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  3.91it/s]
Global seed set to 42


Saved prediction image: test_codec/beauty/im1_pred.png
(512, 512, 3)
(512, 512, 3)
Selected timesteps for ddim sampler: [  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341
 361 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701
 721 741 761 781 801 821 841 861 881 901 921 941 961 981]
Selected alphas for ddim sampler: a_t: tensor([0.9983, 0.9804, 0.9609, 0.9398, 0.9171, 0.8930, 0.8674, 0.8404, 0.8121,
        0.7827, 0.7521, 0.7207, 0.6885, 0.6557, 0.6224, 0.5888, 0.5551, 0.5215,
        0.4882, 0.4552, 0.4229, 0.3913, 0.3605, 0.3308, 0.3023, 0.2750, 0.2490,
        0.2245, 0.2014, 0.1799, 0.1598, 0.1413, 0.1243, 0.1087, 0.0946, 0.0819,
        0.0705, 0.0604, 0.0514, 0.0435, 0.0365, 0.0305, 0.0254, 0.0210, 0.0172,
        0.0140, 0.0113, 0.0091, 0.0073, 0.0058]); a_(t-1): [0.99914998 0.99829602 0.98038077 0.96087277 0.93978298 0.91713792
 0.89298052 0.86737001 0.84038192 0.81210774 0.78265446 0.75214338
 0.72070938 0.68849909 0.65566933 0.62238538

DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  4.01it/s]
Global seed set to 42


...

Saved prediction image: test_codec/beauty/im17_pred.png
(512, 512, 3)
(512, 512, 3)
Selected timesteps for ddim sampler: [  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341
 361 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701
 721 741 761 781 801 821 841 861 881 901 921 941 961 981]
Selected alphas for ddim sampler: a_t: tensor([0.9983, 0.9804, 0.9609, 0.9398, 0.9171, 0.8930, 0.8674, 0.8404, 0.8121,
        0.7827, 0.7521, 0.7207, 0.6885, 0.6557, 0.6224, 0.5888, 0.5551, 0.5215,
        0.4882, 0.4552, 0.4229, 0.3913, 0.3605, 0.3308, 0.3023, 0.2750, 0.2490,
        0.2245, 0.2014, 0.1799, 0.1598, 0.1413, 0.1243, 0.1087, 0.0946, 0.0819,
        0.0705, 0.0604, 0.0514, 0.0435, 0.0365, 0.0305, 0.0254, 0.0210, 0.0172,
        0.0140, 0.0113, 0.0091, 0.0073, 0.0058]); a_(t-1): [0.99914998 0.99829602 0.98038077 0.96087277 0.93978298 0.91713792
 0.89298052 0.86737001 0.84038192 0.81210774 0.78265446 0.75214338
 0.72070938 0.68849909 0.65566933 0.6223853

DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  3.93it/s]
Global seed set to 42


Saved prediction image: test_codec/beauty/im18_pred.png
(512, 512, 3)
(512, 512, 3)
Selected timesteps for ddim sampler: [  1  21  41  61  81 101 121 141 161 181 201 221 241 261 281 301 321 341
 361 381 401 421 441 461 481 501 521 541 561 581 601 621 641 661 681 701
 721 741 761 781 801 821 841 861 881 901 921 941 961 981]
Selected alphas for ddim sampler: a_t: tensor([0.9983, 0.9804, 0.9609, 0.9398, 0.9171, 0.8930, 0.8674, 0.8404, 0.8121,
        0.7827, 0.7521, 0.7207, 0.6885, 0.6557, 0.6224, 0.5888, 0.5551, 0.5215,
        0.4882, 0.4552, 0.4229, 0.3913, 0.3605, 0.3308, 0.3023, 0.2750, 0.2490,
        0.2245, 0.2014, 0.1799, 0.1598, 0.1413, 0.1243, 0.1087, 0.0946, 0.0819,
        0.0705, 0.0604, 0.0514, 0.0435, 0.0365, 0.0305, 0.0254, 0.0210, 0.0172,
        0.0140, 0.0113, 0.0091, 0.0073, 0.0058]); a_(t-1): [0.99914998 0.99829602 0.98038077 0.96087277 0.93978298 0.91713792
 0.89298052 0.86737001 0.84038192 0.81210774 0.78265446 0.75214338
 0.72070938 0.68849909 0.65566933 0.6223853

DDIM Sampler: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  3.93it/s]


Saved prediction image: test_codec/beauty/im19_pred.png
